In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import the dataset
df = pd.read_csv('/kaggle/input/global-aviation-hub-dataset-of-airports-worldwide/airports .csv')

In [ ]:
#show the top 5 records
df.head()

In [ ]:
#shape of dataset
df.shape

# Data checks to perforn

In [ ]:
# Check Missing Values
df.isna().sum()

In [ ]:
# check Duplicates

df.duplicated().sum()


In [ ]:
# check Datatypes
df.info()

In [ ]:
# check the number of unique values of each column
df.nunique()

In [ ]:
# check statistics of dataset
df.describe(exclude='O')

In [ ]:
df.describe(include='O')

In [ ]:
# define numerical columns 
numerical_features = [feature for feature in df.columns if df[feature].dtype !='O']

#print columns
print('We have {} numerical features : {}'.format(len(numerical_features),numerical_features))

In [ ]:
df[numerical_features].isna().sum()

In [ ]:
## Lets check the percentage of missing values
df[numerical_features].isnull().mean()

In [ ]:
def impute_nan(df,variable,mean):
    df[variable+"_mean"]= df[variable].fillna(mean)

In [ ]:
mean = df.elevation_ft.mean()
mean

In [ ]:
impute_nan(df,'elevation_ft',mean)
df.head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
df['elevation_ft'].plot(kind='kde',ax=ax)
df.elevation_ft_mean.plot(kind='kde',ax=ax,color='red')
lines,labels = ax.get_legend_handles_labels()
plt.legend()

In [ ]:
print(df['elevation_ft'].std())
print(df['elevation_ft_mean'].std())

In [ ]:
def impute_nan(df,variable,median):
    df[variable+"_median"]= df[variable].fillna(median)

In [ ]:
median = df.elevation_ft.median()
median

In [ ]:
impute_nan(df,'elevation_ft',median)
df.head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
df['elevation_ft'].plot(kind='kde',ax=ax)
df.elevation_ft_median.plot(kind='kde',ax=ax,color='red')
lines,labels = ax.get_legend_handles_labels()
plt.legend()

In [ ]:
print(df['elevation_ft'].std())
print(df['elevation_ft_median'].std())

In [ ]:
df['elevation_ft'].dropna().sample()


In [ ]:
df['elevation_ft'].isnull().sum()

In [ ]:
df['elevation_ft'].dropna().sample(df['elevation_ft'].isnull().sum(),random_state=0) # replacing NAN values with random numbers

In [ ]:
def impute_nan(df,variable,median):
    df[variable+"_median"] = df[variable].fillna(median)
    df[variable+"_random"] = df[variable]
    ## it will have the random sample to fill the na
    random_sample = df[variable].dropna().sample(df['elevation_ft'].isnull().sum(),random_state=0)
    ##pandas need to have same index in order to merge the dataset
    random_sample.index = df[df[variable].isnull()].index
    df.loc[df[variable].isnull(),variable+'_random'] = random_sample

In [ ]:
median = df.elevation_ft.median()
median

In [ ]:
impute_nan(df,'elevation_ft',median)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
df['elevation_ft'].plot(kind='kde',ax=ax,color='b',linewidth=4,linestyle=':')
#df.elevation_ft_median.plot(kind='kde',ax=ax,color='red')
df.elevation_ft_random.plot(kind='kde',ax=ax,color='red')
lines,labels = ax.get_legend_handles_labels()
plt.legend()

In [ ]:
df.isna().sum()

In [ ]:
df.boxplot(column='elevation_ft_random')

In [ ]:
sns.distplot(df['elevation_ft_random'].dropna())
plt.show()

In [ ]:
df['elevation_ft_random'].describe()

In [ ]:
### Lets compute the Interquantile range to calculate the boundries
IQR = df.elevation_ft_random.quantile(0.75)-df.elevation_ft_random.quantile(0.25) 
IQR

In [ ]:
upper_boundary = df['elevation_ft_random'].mean() + 3* df['elevation_ft_random'].std()
lower_boundary = df['elevation_ft_random'].mean() - 3* df['elevation_ft_random'].std()
print('Lower Boundary : ',lower_boundary),print('Upper Boundary : ',upper_boundary),print('Mean :',df['elevation_ft'].mean())

In [ ]:
lower_bridge = df['elevation_ft_random'].quantile(0.25) - (IQR*1.5)
upper_bridge = df['elevation_ft_random'].quantile(0.75) + (IQR*1.5)
print('Lower Bridge : ',lower_bridge),print('Upper Bridge : ',upper_bridge)

In [ ]:
# Extreme outliers

lower_bridge = df['elevation_ft_random'].quantile(0.25) - (IQR*3)
upper_bridge = df['elevation_ft_random'].quantile(0.75) + (IQR*3)
print(lower_bridge),print(upper_bridge)

In [ ]:
data = df.copy()

In [ ]:
data.loc[data['elevation_ft_random']>5850].shape

In [ ]:
data.loc[data['elevation_ft_random']>5850,'elevation_ft_random'] = 5850

In [ ]:
data.boxplot(column='elevation_ft_random')

In [ ]:
sns.distplot(data['elevation_ft_random'])
plt.show()

In [ ]:
figure = data.elevation_ft_random.hist(bins=50)
figure.set_title('elevation_ft')
figure.set_xlabel('elevation_ft_random')
figure.set_ylabel('Height')

In [ ]:
#categorical features
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']

#print columns
print('We have {} categorical features : {}'.format(len(categorical_features),categorical_features))

In [ ]:
df[categorical_features].isna().sum().sort_values(ascending=False)


In [ ]:
df['iso_country'].value_counts().head(60)

In [ ]:
## to be continued